In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# !pip install -U sentence-transformers
!pip install -U transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 4.2 MB/s 
     |████████████████████████████████| 182 kB 60.4 MB/s 
     |████████████████████████████████| 7.6 MB 41.1 MB/s 


In [3]:
import os
import torch
from torch import nn
import torch.optim as optim

from torch.utils.data import DataLoader
from torchvision import datasets, transforms
# from sentence_transformers import SentenceTransformer
from transformers import BertTokenizer

import pandas as pd
import numpy as np
import tqdm, glob

from torch.utils.data import Dataset, DataLoader
from torchvision.models.feature_extraction import create_feature_extractor

import random


In [4]:
import importlib

In [5]:
%cd /content/drive/MyDrive/multi-modal-music-genre-classification/
import utils

/content/drive/.shortcut-targets-by-id/199GHGZBtQOuthTQ8c7i5neHpvqm7epIw/multi-modal-music-genre-classification


In [6]:
utils = importlib.reload(utils)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

specs, tokens, attention_mask, genre_list, y, music_ids= utils.load_data(tokenizer)
NUM_CLASSES = len(genre_list)
# print(NUM_CLASSES)
# print(len(zipped_data))

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

loading data!
PreTrainedTokenizer(name_or_path='bert-base-uncased', vocab_size=30522, model_max_len=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})


100%|██████████| 91/91 [02:21<00:00,  1.56s/it]


(8622,) (8622, 512) (8622, 512) (8622,)


In [8]:
len(music_ids)

8621

In [9]:
original_test_ids = np.load("test_tokens.npy") #this is after shuffling and saving

In [10]:
count = 0
ids = []
for sentence in original_test_ids:
  for i, token in enumerate(tokens):
    if (token==sentence).all():
      count+=1
      ids.append(i)
      break

In [11]:
count, len(tokens), len(ids)

(863, 8621, 863)

In [12]:
music_ids_in_test = music_ids[ids] #ids are in order of the shuffled test tokens

In [15]:
np.save("test_music_ids.npy", music_ids_in_test)

In [16]:
df_metadata = pd.read_parquet('/content/drive/MyDrive/multi-modal-music-genre-classification/4mula_metadata.parquet')
id_lyrics = df_metadata[['music_id', 'music_lyrics', 'main_genre']]

In [24]:
id_lyrics[id_lyrics['music_id']=='3ade68b8g1e9e60b3']

,music_id,music_lyrics,main_genre
4005,3ade68b8g1e9e60b3,There's so many around the world\nWith voices ...,Gospel/Religious


In [ ]:
n = len(specs)
ids = [i for i in range(n)]
random.shuffle(ids)

#### train val test split 8:1:1
train_ids = ids[0:int(0.8*n)]
val_ids = ids[int(0.8*n):int(0.9*n)]
test_ids = ids[int(0.9*n):]

In [ ]:
train_specs, train_tokens, train_attention_mask, train_y = specs[train_ids], tokens[train_ids], attention_mask[train_ids], y[train_ids]

np.save("train_tokens.npy", train_tokens, allow_pickle=True)
np.save("train_attention_masks.npy", train_attention_mask, allow_pickle=True)
np.save("genre_list.npy", genre_list)
np.save("train_specs.npy", train_specs, allow_pickle=True)
np.save("train_y.npy", train_y)

In [ ]:
val_specs, val_tokens, val_attention_mask, val_y = specs[val_ids], tokens[val_ids], attention_mask[val_ids], y[val_ids]

np.save("val_tokens.npy", val_tokens, allow_pickle=True)
np.save("val_attention_masks.npy", val_attention_mask, allow_pickle=True)
np.save("val_specs.npy", val_specs, allow_pickle=True)
np.save("val_y.npy", val_y)

In [ ]:
test_specs, test_tokens, test_attention_mask, test_y = specs[test_ids], tokens[test_ids], attention_mask[test_ids], y[test_ids]

np.save("test_tokens.npy", test_tokens, allow_pickle=True)
np.save("test_attention_masks.npy", test_attention_mask, allow_pickle=True)
np.save("test_specs.npy", test_specs, allow_pickle=True)
np.save("test_y.npy", test_y)